In [1]:
time_start = 20150101
time_end =  20210630
today = 20211130
feat_list = ['pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']
return_value = '5d_ret'
output_path = '/home/wuwenjun/Data/AlphaNet_Original_Input/'
day_back = 5 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天

In [9]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

In [3]:
dataframe_list = DataAPI.get_stock_prices(start_date=time_start, end_date=time_end).reset_index().set_index(['timestamp','ticker'])

In [4]:
dataframe_list = dataframe_list[feat_list].reset_index()
time_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
time_df['timestamp_shift'] = time_df['timestamp'].apply(lambda x : DataAPI.get_next_trading_day(x, output='datetime'))
dataframe_list = pd.merge(left = dataframe_list , right = time_df , how = 'left',on = 'timestamp')
dataframe_list.drop(['timestamp'],axis=1,inplace=True)
dataframe_list.rename(columns = {"timestamp_shift":"timestamp"},inplace=True)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp']) + pd.Timedelta(hours=9,minutes = 30)
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 000001      15.84   15.99   16.28   15.60   16.02   
                    000002      13.90   14.39   15.29   14.22   14.91   
                    000004      15.57   15.58   15.90   15.30   15.69   
                    000005       4.10    4.10    4.10    4.10    4.10   
                    000006       7.05    7.10    7.39    7.02    7.08   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688700      46.60   47.80   54.50   47.03   53.38   
                    688777      90.00   89.95   94.80   89.06   94.15   
                    688788     138.20  137.28  148.50  136.33  145.38   
                    688819      43.68   43.40   44.98   43.31   44.49   
                    688981      59.25   59.69   62.50   59.40   61.82   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 000001   15.9605  2860436.43   1.1364    2.9079   
                    000002   14.7858  6560835.70   7.2662    6.7834   
                    000004   15.5709    28095.12   0.7707    3.3496   
                    000005    4.1000        0.00   0.0000    0.0000   
                    000006    7.1881  1078647.12   0.4255    8.0562   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688700   50.7153    64336.83  14.5494   19.2222   
                    688777   92.9798    19001.61   4.6111    4.6288   
                    688788  144.3683    13572.56   5.1954    7.7977   
                    688819   44.3727    47249.01   1.8544    4.6754   
                    688981   61.5420   777950.53   4.3376    7.1003   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 000001         5.7657  0.18  
                    000002         8.1741  1.01  
                    000004         4.5343  0.12  
                    000005         0.0000  0.00  
                    000006        13.2751  0.03  
...                                   ...   ...  
2021-07-01 09:30:00 688700        19.2222  6.78  
                    688777         4.6288  4.15  
                    688788         7.7977  7.18  
                    688819         4.6754  0.81  
                    688981         7.1003  2.57  

[5357761 rows x 11 columns]

In [5]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/factor_lib")

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Process ForkPoolWorker-131:
Process ForkPoolWorker-102:
Process ForkPoolWorker-124:
Process ForkPoolWorker-135:
Process ForkPoolWorker-128:
Process ForkPoolWorker-142:
Process ForkPoolWorker-136:
Process ForkPoolWorker-140:
Process ForkPoolWorker-134:
Process ForkPoolWorker-93:
Process ForkPoolWorker-126:
Process ForkPoolWorker-123:
Process ForkPoolWorker-139:
Process ForkPoolWorker-104:
Process ForkPoolWorker-137:
Process ForkPoolWorker-101:
Process ForkPoolWorker-94:
Process ForkPoolWorker-122:
Process ForkPoolWorker-133:
Process ForkPoolWorker-141:
Process ForkPoolWorker-92:
Process ForkPoolWorker-120:
Process ForkPoolWorker-125:
Process ForkPoolWorker-130:
Process ForkPoolWorker-86:
Process ForkPoolWorker-83:
Process ForkPoolWorker-107:
Process ForkPoolWorker-129:
Process ForkPoolWorker-95:
Process ForkPoolWorker-88:
Process ForkPoolWorker-105:
Process ForkPoolWorker-138:
Process ForkPoolWorker-108:
Process ForkPoolWorker-99:
Process ForkPoolWorker-87:
Process ForkPoolWorker-144:
P

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-115:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.p

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessi

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/minico

KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._sem

In [6]:
FT.load_feature_from_dataframe(dataframe_list,universe='Investable')
FT.load_return_data()

Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:07<00:00, 215.63it/s]


In [7]:
data_info = FT.data_info

In [8]:
# Drop 掉不需要的return 列
df = pd.DataFrame()
return_list = feat_list.copy()
return_list.append(return_value)
for timestamp in tqdm(data_info.keys()):
    timestamp = pd.to_datetime(timestamp)
    one_data = data_info[timestamp][return_list].reset_index()
    one_data['timestamp'] = timestamp
    df = pd.concat([df,one_data],axis=0)
df.sort_values(by = ['timestamp','ticker'],inplace=True)
display(df)

100%|██████████| 1580/1580 [05:06<00:00,  5.15it/s]


,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret,timestamp
0,000001,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710,2015-01-06 09:30:00
1,000002,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164,2015-01-06 09:30:00
2,000004,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390,2015-01-06 09:30:00
3,000006,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571,2015-01-06 09:30:00
4,000007,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844,2015-01-06 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476,2021-07-01 09:30:00
4025,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133,2021-07-01 09:30:00
4026,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721,2021-07-01 09:30:00
4027,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164,2021-07-01 09:30:00


In [ ]:
df.set_index(['timestamp','ticker']).to_csv(output_path + 'data.csv')
# Save_TimeStamp
df['timestamp'].drop_duplicates()

In [ ]:
for ticker in tqdm(df['ticker'].drop_duplicates()):
    dict_output = {}
    one_data = df[df['ticker'] == ticker]
    for i in range(0,one_data.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        time_stamp = str(pd.to_datetime(one_data.iloc[i+day-1]['timestamp']))
        x = one_data.iloc[i:i+day,:]
        y = one_data.iloc[i+day-1][return_value]
        x.set_index(['ticker','timestamp'],inplace=True)
        dict_output[time_stamp] = [x,y]
    np.save(output_path + ticker, dict_output)

 60%|██████    | 2580/4276 [49:04<27:03,  1.04it/s]  

In [15]:
dict_output

{Timestamp('2015-01-06 09:30:00'): {},
 Timestamp('2015-01-07 09:30:00'): {},
 Timestamp('2015-01-08 09:30:00'): {},
 Timestamp('2015-01-09 09:30:00'): {},
 Timestamp('2015-01-12 09:30:00'): {},
 Timestamp('2015-01-13 09:30:00'): {},
 Timestamp('2015-01-14 09:30:00'): {},
 Timestamp('2015-01-15 09:30:00'): {},
 Timestamp('2015-01-16 09:30:00'): {},
 Timestamp('2015-01-19 09:30:00'): {},
 Timestamp('2015-01-20 09:30:00'): {},
 Timestamp('2015-01-21 09:30:00'): {},
 Timestamp('2015-01-22 09:30:00'): {},
 Timestamp('2015-01-23 09:30:00'): {},
 Timestamp('2015-01-26 09:30:00'): {},
 Timestamp('2015-01-27 09:30:00'): {},
 Timestamp('2015-01-28 09:30:00'): {},
 Timestamp('2015-01-29 09:30:00'): {},
 Timestamp('2015-01-30 09:30:00'): {},
 Timestamp('2015-02-02 09:30:00'): {},
 Timestamp('2015-02-03 09:30:00'): {},
 Timestamp('2015-02-04 09:30:00'): {},
 Timestamp('2015-02-05 09:30:00'): {},
 Timestamp('2015-02-06 09:30:00'): {},
 Timestamp('2015-02-09 09:30:00'): {},
 Timestamp('2015-02-10 09

In [17]:
# Save

np.save('/home/wuwenjun/Data/AlphaNet_Original_Input.npy', dict_output) 

# Load
# read_dictionary = np.load('my_file.npy').item()
# print(read_dictionary['hello']) 

KeyboardInterrupt: 

In [ ]:
dataframe_list.to_csv('/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))
print("save to " + '/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))